# Week 11 Homework - Getting data


1. Load the dataframe saved from this week's lab.
1. Scrape data from this page https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)
1. Append to the dataframe: projected GDP (nominal) from IMF estimates from 2022 to 2027. (If data is not available for the country use None)
1. Add a new column and calculate average increase in terms of percentage. 
1. Sort by this column in descending order.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False

In [21]:
def avg_increase(val1, val2, val3, val4, val5, val6):
    if(isnan(val1) | isnan(val2) | isnan(val3) | isnan(val4) | isnan(val5) | isnan(val6)):
        return 0
    percent = ((val2-val1)/val1 + (val3-val2)/val2 + (val4-val3)/val3 + (val5-val4)/val4 + (val6-val5)/val5)*20
    return percent

In [16]:
country = pd.read_csv("country.csv")
country.drop(columns=country.columns[0], 
        axis=1, 
        inplace=True)
country['Country'] = country['Country'].str.strip()
country.head()

,Country,Capital,Population,Area
0,Andorra,Andorra la Vella,84000,468.0
1,United Arab Emirates,Abu Dhabi,4975593,82880.0
2,Afghanistan,Kabul,29121286,647500.0
3,Antigua and Barbuda,St. John's,86754,443.0
4,Anguilla,The Valley,13254,102.0


In [17]:
page_content = pd.read_html("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)")[5]
page_content.drop(columns=['2020','2021'], 
        axis=1, 
        inplace=True)
page_content.rename(columns={'Country (or dependent territory)':'Country'}, 
        inplace=True)
page_content

,Country,2022,2023,2024,2025,2026,2027
0,Afghanistan,20136,20136,20136,20136,20136,20136
1,Albania,17942,19062,20339,21622,22931,24249
2,Algeria,193601,199382,204551,212973,222863,231837
3,Andorra,3400,3580,3755,3913,4072,4225
4,Angola,124862,133335,140706,150343,159450,167503
...,...,...,...,...,...,...,...
191,Vietnam,408947,462644,515607,571117,629625,690109
192,Palestine,19698,20891,21853,22745,23634,24603
193,Yemen,28134,29556,31233,33196,35491,38197
194,Zambia,26665,27709,29865,31987,34014,36350


In [22]:
merged = pd.merge(country,page_content, on='Country', how= 'left')
avg_percent_inc = []
for index, row in merged.iterrows():
    avg_percent_inc.append(round(avg_increase(row['2022'],row['2023'],row['2024'],row['2025'],row['2026'],row['2027']),5))
merged['AvgPercentIncrease'] = avg_percent_inc
merged = merged.replace({np.nan: None})
merged.sort_values(by = 'AvgPercentIncrease', ascending=False)

,Country,Capital,Population,Area,2022,2023,2024,2025,2026,2027,AvgPercentIncrease
207,South Sudan,Juba,8260490,644329.0,5730.0,10771.0,11159.0,11161.0,11652.0,12320.0,20.34558
163,Nigeria,Abuja,154000000,923768.0,510588.0,580121.0,656692.0,744225.0,844414.0,958491.0,13.42371
234,Uzbekistan,Tashkent,27865738,447400.0,73060.0,81809.0,93656.0,105252.0,117433.0,130800.0,12.35874
161,Niger,Niamey,15878271,1267000.0,15620.0,17303.0,19910.0,23268.0,25596.0,27762.0,12.23494
146,Mongolia,Ulan Bator,3086918,1565000.0,18102.0,21047.0,23679.0,26415.0,28848.0,31034.0,11.42343
...,...,...,...,...,...,...,...,...,...,...,...
190,Russia,Moscow,140702000,17100000.0,1829050.0,1713154.0,1718194.0,1742194.0,1766044.0,1795503.0,-0.32167
122,Kuwait,Kuwait City,2789132,17820.0,186610.0,180665.0,176640.0,176552.0,179072.0,183101.0,-0.35724
135,Libya,Tripoli,6461454,1759540.0,48773.0,45195.0,43618.0,42975.0,43003.0,42999.0,-2.44873
87,Equatorial Guinea,Malabo,1014999,28051.0,16335.0,14956.0,13083.0,12975.0,12782.0,13071.0,-4.20348
